In [1]:
import torch
torch.manual_seed(0)
dropout = torch.nn.Dropout(p=0.5)

X = torch.ones(size=(3, 10), requires_grad=True)
output = dropout(X)
output.retain_grad()

print("output:", output)
print("X:", X)
output.backward(gradient=torch.ones_like(output))
print("output grad after backprop:", output.grad)
print("X grad after backprop:", X.grad)

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


output: tensor([[2., 0., 0., 2., 2., 0., 0., 2., 2., 2.],
        [2., 0., 0., 2., 2., 2., 0., 2., 0., 0.],
        [0., 0., 2., 2., 2., 2., 2., 0., 0., 0.]], grad_fn=<MulBackward0>)
X: tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], requires_grad=True)
output grad after backprop: tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
X grad after backprop: tensor([[2., 0., 0., 2., 2., 0., 0., 2., 2., 2.],
        [2., 0., 0., 2., 2., 2., 0., 2., 0., 0.],
        [0., 0., 2., 2., 2., 2., 2., 0., 0., 0.]])


In [2]:
from jax import numpy as jnp
import numpy as np
import brunoflow as bf

np.random.seed(0)

dropout_bf = bf.net.Dropout(p=0.5)
dropout_bf.training = True
X = bf.Node(jnp.ones(shape=(3, 10)), name="x")

output = dropout_bf(X)
output.zero_gradients()

print("output:", output)
print("X:", X)
output.backprop()
print("output grad after backprop:", output)
print("X grad after backprop:", X) # note that the grad is only non-zero for the values that were not dropped out (matches value of output)


output: node(name: *, val: [[2. 2. 2. 2. 0. 2. 0. 2. 2. 0.]
 [2. 2. 2. 2. 0. 0. 0. 2. 2. 2.]
 [2. 2. 0. 2. 0. 2. 0. 2. 2. 0.]], grad: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]])
X: node(name: x, val: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], grad: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]])
output grad after backprop: node(name: *, val: [[2. 2. 2. 2. 0. 2. 0. 2. 2. 0.]
 [2. 2. 2. 2. 0. 0. 0. 2. 2. 2.]
 [2. 2. 0. 2. 0. 2. 0. 2. 2. 0.]], grad: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]])
X grad after backprop: node(name: x, val: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], grad: [[2. 2. 2. 2. 0. 2. 0. 2. 2. 0.]
 [2. 2. 2. 2. 0. 0. 0. 2. 2. 2.]
 [2. 2. 0. 2. 0. 2. 0. 2. 2. 0.]])


In [31]:
from brunoflow.net.mlp import MLP
np.random.seed(0)
m = MLP(10, 5, 2, dropout_prob=0.8)
m.eval()
inputs = bf.Node(np.random.normal(size=(3, 10)), name="x")
m(inputs).backprop()
np.mean(inputs.compute_entropy().val)

DeviceArray(1.0579184, dtype=float32)

<module 'brunoflow.net.mlp' from '/home/kevin/code/rycolab/brunoflow/brunoflow/net/mlp.py'>